# Projeto 4- Classificação Binária Breast Cancer - classificar somente um registro e salvar o classificador

## Importação de bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
from google import colab as cl

torch.__version__

'2.0.1+cu118'

## Base de dados

In [2]:
np.random.seed(123)
torch.manual_seed(123)

In [3]:
file_upload = cl.files.upload() #importando arquivo para computador

previsores = pd.read_csv('/content/entradas_breast.csv')
classe = pd.read_csv('/content/saidas_breast.csv')

Saving saidas_breast.csv to saidas_breast.csv
Saving entradas_breast.csv to entradas_breast.csv


In [4]:
previsores = torch.tensor(np.array(previsores), dtype=torch.float)
classe = torch.tensor(np.array(classe), dtype=torch.float)

## Transformação dos dados para tensores

In [5]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores, classe), batch_size=10, shuffle=True)

## Construção do modelo

In [6]:
class classificador_torch(nn.Module):

  def __init__(self):
    super().__init__()

    #30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(in_features=30, out_features=8)
    torch.nn.init.normal_(self.dense0.weight, mean=0.0, std=0.05)

    self.dense1 = nn.Linear(in_features=8, out_features=8)
    torch.nn.init.normal_(self.dense1.weight, mean=0.0, std=0.05)

    self.dense2 = nn.Linear(in_features=8, out_features=1)
    self.activation = nn.ReLU()
    self.dropout = nn.Dropout(0.2)
    self.output = nn.Sigmoid()


  def forward(self, X):
    X = self.dense0(X)
    X = self.activation(X)
    X = self.dropout(X)

    X = self.dense1(X)
    X = self.activation(X)
    X = self.dropout(X)

    X = self.dense2(X)
    X = self.output(X)

    return X

In [7]:
classificador = classificador_torch()

In [8]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(classificador.parameters(), lr=0.001, weight_decay=0.0001)

## Treinamento do modelo

In [9]:
for epoch in range(100):
  running_loss = 0. #float
  for data in train_loader:

    inputs, labels = data

    optimizer.zero_grad() #zerando gradiente

    outputs = classificador(inputs)

    loss = criterion(outputs, labels) #cálculo do erro

    loss.backward() #backpropagation

    optimizer.step() #atualizando pesos

    running_loss += loss.item() # somando loss

  print('Epoch %3d: Loss %.5f' % (epoch + 1, running_loss/len(train_loader) ))



Epoch   1: Loss 1.65405
Epoch   2: Loss 0.83266
Epoch   3: Loss 0.68309
Epoch   4: Loss 0.61071
Epoch   5: Loss 0.58727
Epoch   6: Loss 0.59212
Epoch   7: Loss 0.54197
Epoch   8: Loss 0.52175
Epoch   9: Loss 0.51248
Epoch  10: Loss 0.51385
Epoch  11: Loss 0.50309
Epoch  12: Loss 0.47485
Epoch  13: Loss 0.47657
Epoch  14: Loss 0.45629
Epoch  15: Loss 0.44756
Epoch  16: Loss 0.44154
Epoch  17: Loss 0.41747
Epoch  18: Loss 0.39531
Epoch  19: Loss 0.42316
Epoch  20: Loss 0.37898
Epoch  21: Loss 0.40490
Epoch  22: Loss 0.38421
Epoch  23: Loss 0.38859
Epoch  24: Loss 0.39575
Epoch  25: Loss 0.37081
Epoch  26: Loss 0.37289
Epoch  27: Loss 0.36106
Epoch  28: Loss 0.37193
Epoch  29: Loss 0.37447
Epoch  30: Loss 0.35007
Epoch  31: Loss 0.36657
Epoch  32: Loss 0.31675
Epoch  33: Loss 0.32431
Epoch  34: Loss 0.33630
Epoch  35: Loss 0.36814
Epoch  36: Loss 0.35728
Epoch  37: Loss 0.33705
Epoch  38: Loss 0.31823
Epoch  39: Loss 0.32361
Epoch  40: Loss 0.33200
Epoch  41: Loss 0.32638
Epoch  42: Loss 

## Classificar somente um registro

In [10]:
novo = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [11]:
classificador.eval()

classificador_torch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [12]:
previsao = classificador(novo)

In [13]:
previsao

tensor([[1.]], grad_fn=<SigmoidBackward0>)

In [14]:
previsao = previsao.detach() #buscando os valores do tensor

In [15]:
previsao

tensor([[1.]])

In [16]:
previsao = previsao.numpy()

In [17]:
type(previsao)

numpy.ndarray

In [18]:
previsao

array([[1.]], dtype=float32)

In [19]:
previsao = (previsao > 0.5)
previsao

array([[ True]])

## Salvar o classificador

In [20]:
classificador.state_dict()

OrderedDict([('dense0.weight',
              tensor([[-6.3329e-01, -6.4088e-03, -3.8681e-01, -1.0687e-01,  9.4179e-02,
                       -2.8071e-02,  5.8217e-02, -5.5828e-02,  1.4919e-02,  5.5338e-02,
                        8.5171e-03, -1.0727e-03,  3.1089e-03, -1.7085e-02,  3.1640e-02,
                        2.3500e-01, -3.9340e-01, -1.7852e-01, -6.7033e-02,  5.4838e-01,
                       -4.7472e-01,  2.4388e-02, -3.2421e-01,  1.8188e-01,  1.4345e-02,
                        3.0411e-02,  1.2222e-02,  4.2423e-02,  1.6629e-02,  6.2451e-01],
                      [ 9.6147e-02,  8.9823e-03,  6.2657e-02, -2.7833e-02,  2.5520e-02,
                       -1.4683e-02,  8.4852e-09, -5.5642e-02, -4.3206e-02,  2.9459e-04,
                        3.3087e-02,  1.5457e-03, -3.6564e-02,  7.3219e-02, -9.4868e-11,
                       -5.2539e-07,  3.7381e-03,  2.0600e-09,  6.6002e-09,  2.5737e-10,
                       -7.0341e-02,  3.3297e-02, -3.6295e-02, -3.3981e-02, -7.6689e-02,


In [21]:
torch.save(classificador.state_dict(), 'checkpoint.pth')